## Usando o detectores_json_k_dobras no Colab

Este notebook executa somente o programa **experimento.py** responsável pelo treinamento e teste das redes profundas no exemplo de Eucaliptos que já vem disponível no detectores_json_k_dobras. 

A preparação de outros dados, antes de rodar **experimento.py**, e a geração dos gráficos, posteriormente, não dependem de GPU e devem ser executados na sua própria máquina, seguindo as instruções do README.md de detectores_json_k_dobras. 



## Preparando o código

* Crie a pasta checkpoints dentro de detectores_json_k_dobras e coloque lá os arquivos .pth das redes que você pretende usar (leia no README.md do detectores_json_k_dobras a seção "Escolhendo as arquiteturas a serem testadas"). 

* Copie a pasta com todos os arquivos do detectores_json_k_dobras (incluindo os checkpoints) para sua conta no Google Drive (+ NOVO -> Upload de Pasta). Como tem um exemplo pesado com imagens de Eucalipto, pode demorar um pouco para subir. Se não quiser usar este exemplo, apague ele da sua máquina antes de subir.

* Rode a linha abaixo para que o Colab possa começar a acessar os seus dados no Drive (o Drive vai pedir que você autorize o acesso e dependendo do tipo de conta vai também gerar um código longo para você copiar no Colab, leia e siga as orientações).



In [2]:
# Liga o Colab à sua conta no Drive
from google.colab import drive
drive.mount('/content/drive')

# Entra na pasta do detectores_json_k_dobras que você subiu
# Se subiu em outro lugar, ajuste o comando abaixo
%cd /content/drive/MyDrive/detectores_json_k_dobras

# Lista o conteúdo da pasta para conferir se está tudo OK
!ls 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/detectores_json_k_dobras
checkpoints    experimento.py  mmdetection		      README.md  utils
conda_init.sh  graficos.R      MMdetection_Colab_Drive.ipynb  roda.sh
dataset        install.sh      MMDet_Tutorial.ipynb	      Teste.py


# Truque para evitar que o Colab desconecte 

Depois de muito tempo sem interação o Colab pode desconectar. Use F12 para abrir a janela de código do Google Chrome. Clique em Console (Veja uma aba na parte de baixo desta janela de código). Copie o código abaixo para a linha de comando do console e tecle ENTER:

-----

function ClickConnect(){
  console.log("Connnect Clicked - Start"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
  console.log("Connnect Clicked - End"); 
};
setInterval(ClickConnect, 60000)

-----

Link explicando este truque: https://www.ti-enxame.com/pt/python/como-impedir-que-o-google-colab-se-desconecte/810821538/


## Instalando os pacotes necessários

Na primeira vez que instala pode demorar uns 20 minutos (ou mais) para instalar tudo. Não se desespere !!! Para rodar o experimento geralmente demora MUITO MAIS :-) 


#### Instala o pytorch e o mmcv

In [3]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==1.3.5



Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4 MB 1.3 kB/s 
     |████████████████████████████████| 6.6 MB 690 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.1+cu101 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.5.1+cu101 which is incompatible.
     |████████████████████████████████| 303 kB 12.3 MB/s 
     |█████████████████

### Ajusta o mmdetection

Dentro do detectores_json_k_dobras tem uma versão do mmdetection que não é a última, mas que estava funcionando para nós (se quiser tentar a última descomente os comandos rm e git abaixo)

PS: Se pedir para "Restart Runtime" depois que instalar a versão mais antiga do Pillow, pode clicar sem medo.

In [4]:
# Se quiser instalar a última versão do mmdetection descomente
# as linhas abaixo (por usa conta e risco)
# !rm -rf mmdetection
# !git clone https://github.com/open-mmlab/mmdetection.git

# Entra na pasta do mmdetection e ajusta as coisas
%cd mmdetection
!pip install -e .
# Volta para a pasta raiz do detectores_json_k_dobras
%cd ..

# Parece que o Colab precisa desta versão do Pillow 
!pip install Pillow==7.0.0



/content/drive/MyDrive/detectores_json_k_dobras/mmdetection
Obtaining file:///content/drive/MyDrive/detectores_json_k_dobras/mmdetection
  Running setup.py develop for mmdet
/content/drive/MyDrive/detectores_json_k_dobras
     |████████████████████████████████| 2.1 MB 12.3 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bokeh 2.3.3 requires pillow>=7.1.0, but you have pillow 7.0.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


## Testa a instalação

In [5]:
# Verificando instalação do Pytorch 
import torch, torchvision
print("Versão do Torch instalada: ",torch.__version__)
print("Achou o CUDA para usar a GPU: ", torch.cuda.is_available())

# Verificando instalação do mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print("Versão do CUDA usada no MMCV: ", get_compiling_cuda_version())
print("Versão do compilador usado no MMCV: ",get_compiler_version())

# Verificando instalação do MMDetection
import mmdet
print(mmdet.__version__)



Versão do Torch instalada:  1.5.1+cu101
Achou o CUDA para usar a GPU:  True


ModuleNotFoundError: ignored

## Rodando o experimento

O comando abaixo é que roda o experimento (que depende de GPU). Quando terminar, faça o Download da pasta dataset e rode na sua máquina o comando abaixo para gerar os gráficos (você pode também rodar este arquivo usando o Rstudio). Leia o README.md para saber onde os resultados são gravados (assim não precisa fazer o Download de tudo, apenas o que foi gerado no experimento)

$ Rscript graficos.R

In [ ]:
# Muda novamente para a pasta correta
%cd /content/drive/MyDrive/detectores_json_k_dobras

# Roda o experimento
!python experimento.py

/content/drive/MyDrive/detectores_json_k_dobras
Torch:  1.5.1+cu101 True
mmdet:  2.12.0
mmvc:  1.3.5
cuda-version:  11.1
compiler:  GCC 7.5
python:  3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]
Pasta do mmdetection:  /content/drive/MyDrive/detectores_json_k_dobras/mmdetection
Pasta com o dataset:  /content/drive/MyDrive/detectores_json_k_dobras/dataset
Pasta com os checkpoints (*.pth):  /content/drive/MyDrive/detectores_json_k_dobras/checkpoints
Arquiteturas que serão testadas:
{'retinanet': {'config_file': 'configs/retinanet/retinanet_r50_fpn_1x_coco.py', 'checkpoint': '/content/drive/MyDrive/detectores_json_k_dobras/checkpoints/retinanet_r50_fpn_1x_coco_20200130-c2398f9e.pth'}, 'faster': {'config_file': 'configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py', 'checkpoint': '/content/drive/MyDrive/detectores_json_k_dobras/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'}, 'atss': {'config_file': 'configs/atss/atss_r50_fpn_1x_coco.py', 'checkpoint': '/content/drive/M